# Email Service - Send to all consumers


## Download dependencies


In [140]:
%pip install openai bs4 pandas requests smtp_email_sender smtplib

ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib


In [127]:
!pip install python-docx

## Prompt engineering

This will be used to guide LLM


### Zero shot prompt for text classification

In [145]:
text = """
# Identity
You are AI model that classifies text into one of the 3 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 
The 3 categories are **Immigration**, **Economy**, **Civil**

# Instructions
* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 3 categories mentioned
* Never answer in a sentence. 
* Respond using EXACTLY one word from the allowed categories.
"""

with open('political_text_classifier.txt', 'w') as f:
    f.write(text)

### Few shot prompt for summarizer 

In [146]:
text = """
# Identity
You are an AI summarizing tool that explains political legislation in clear, easy-to-understand language so the public can better understand the text.

# Instructions
* Provide clear, simple, and accurate explanations.
* Use bullet points only.
* Focus only on what the legislation contains.
* Ignore information such as authors, dates, or political commentary.
* Do not add opinions, warnings, or analysis — only summarize content.
* Bullet points must be detailed enough for people to extract meaningful insights.

# Few Shot Examples

### Use the following examples to guide your summaries.

## USER
Summarize the following legislation:
"The bill establishes a statewide program for distributing emergency cooling units to residents during high-temperature events. It directs the health department to manage distribution, prioritize vulnerable populations, and create public guidance on extreme-heat safety."

## ASSISTANT
* Creates a statewide program to provide emergency cooling devices during periods of extreme heat.
* Assigns the state health department to oversee how the devices are distributed.
* Requires the department to focus distribution on groups considered vulnerable to heat-related risks.
* Instructs the department to produce public information explaining how to stay safe during high-temperature events.

## USER
Summarize the following legislation:
"The act requires all state agencies to publish annual cybersecurity readiness reports. It also mandates the creation of minimum cybersecurity standards and establishes a review board to evaluate agency compliance."

## ASSISTANT
* Requires every state agency to publish a yearly report describing its cybersecurity preparedness.
* Establishes baseline cybersecurity standards that all agencies must follow.
* Creates a review board responsible for checking whether agencies meet the required cybersecurity standards.

## USER
Summarize the following legislation: [INSERT LEGISLATION HERE]

## ASSISTANT
Generate bullet points summarizing the legislation above.
"""

with open('political_text_summarizer.txt', 'w') as f:
    f.write(text)

## Load prompts


In [130]:
with open("political_text_classifier.txt", "r", encoding="utf-8") as file:
    political_text_classifier = file.read()

In [131]:
with open("political_text_summarizer.txt", "r", encoding="utf-8") as file:
    political_text_summarizer = file.read()

## Define AI functions

This uses OpenAI SDK and GPT 4o as base LLM


### Set up OpenAI client


In [148]:
from openai import OpenAI

# Set api key to environment variables
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### Political text classification


In [ ]:
def classifyText(input_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": political_text_classifier
            },
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip().split()[0]

### Summarizer of legislative text


In [ ]:
def summarizeText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": political_text_summarizer},
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip()

## Build webscraper to get NYC legislation

BeautifulSoup will be used to access HTML of NYC Legistar website. We will save the scraped data in a Pandas dataframe for further processing


### Scrape council committee meetings


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

council_meetings = []

# Skip the header row which is index 0
# Only get the 2 most recent meetings
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')

    committee = cells[0].get_text(strip=True)
    date = cells[1].get_text(strip=True)
    meeting_time = cells[3].get_text(strip=True)
    
    if meeting_time == "Deferred":
        continue
    
    # Get the agenda link (from the 7th column)
    meeting_detail = cells[6].find('a')
    meeting_detail_aspx = meeting_detail['href']    
    
    if len(council_meetings) < 2:
        council_meetings.append({
            'Date': date,
            'Committee': committee,
            'Meeting Details': meeting_detail_aspx
        })
    else:
        break

### View council meetings in a formatted manner


In [ ]:
df = pd.DataFrame(council_meetings)
df

### Scrape Meeting Details + Process through AI

- Iterate through each meeting detail and iterate through each "Introduction" legislative text
- Run the text through the 2 AI helper functions


In [ ]:
categories = {
    "Immigration": [],
    "Economy": [],
    "Civil": []
}

In [ ]:
from io import BytesIO
from docx import Document

for meeting in council_meetings:
    meeting_details_url = f"https://legistar.council.nyc.gov/{meeting['Meeting Details']}"
    meeting_details_html = requests.get(meeting_details_url).text
    soup = BeautifulSoup(meeting_details_html, "html.parser")  
    table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridMain_ctl00')
    legislation_file = []
    
    for tr in table.find_all('tr')[1:]:  # Skip header row
        cells = tr.find_all('td')
        file_type = cells[6].get_text(strip=True)
        if file_type != "Introduction":
            continue
        
        file_locator = cells[0].find('a')['href']
        if len(legislation_file) > 2:
            break
        
        legislation_file.append(file_locator)
    
    for file_locator in legislation_file:
        response = requests.get(f"https://legistar.council.nyc.gov/{file_locator}").text
        soup = BeautifulSoup(response, "html.parser")

        span_body = soup.find('span', id="ctl00_ContentPlaceHolder1_lblAttachments2")
        legislation_pdf_link = span_body.find_all('a')[2]['href']
        name = soup.find('span', id="ctl00_ContentPlaceHolder1_lblName2").get_text(strip=True)
        status = soup.find('span', id="ctl00_ContentPlaceHolder1_lblStatus2").get_text(strip=True)

        fetched_document = requests.get(f"https://legistar.council.nyc.gov/{legislation_pdf_link}")
        doc = Document(BytesIO(fetched_document.content))    
        legislation_text = '\n'.join([para.text for para in doc.paragraphs])

        category = classifyText(legislation_text)
        summarized_category = summarizeText(legislation_text)

        categories[category].append({
            "Name": name,
            "Status": status,
            "Summarized": summarized_category
        })

## Email to subscribers


### Convert the markdown summaries to HTML email format

In [ ]:
def create_html_email(items):    
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                color: #333;
                max-width: 800px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f4f4f4;
            }
            .container {
                background-color: white;
                padding: 30px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            h1 {
                color: #2c3e50;
                border-bottom: 3px solid #3498db;
                padding-bottom: 10px;
            }
            .item {
                margin: 30px 0;
                padding: 20px;
                background-color: #f9f9f9;
                border-left: 4px solid #3498db;
                border-radius: 4px;
            }
            .item-header {
                display: flex;
                justify-content: space-between;
                align-items: center;
                margin-bottom: 15px;
            }
            .item-name {
                font-size: 18px;
                font-weight: bold;
                color: #2c3e50;
                flex: 1;
            }
            .status {
                padding: 5px 15px;
                border-radius: 20px;
                font-size: 14px;
                font-weight: bold;
                text-transform: uppercase;
            }
            .status-passed {
                background-color: #27ae60;
                color: white;
            }
            .status-under-review {
                background-color: #f39c12;
                color: white;
            }
            .status-rejected {
                background-color: #e74c3c;
                color: white;
            }
            .status-default {
                background-color: #95a5a6;
                color: white;
            }
            .summary {
                margin-top: 15px;
                line-height: 1.8;
            }
            .summary h2 {
                color: #2c3e50;
                font-size: 16px;
                margin-top: 15px;
                margin-bottom: 10px;
            }
            .summary h3 {
                color: #34495e;
                font-size: 14px;
                margin-top: 12px;
                margin-bottom: 8px;
            }
            .summary ul {
                margin: 10px 0;
                padding-left: 20px;
            }
            .summary li {
                margin: 5px 0;
            }
            .summary strong {
                color: #2980b9;
            }
            .footer {
                margin-top: 40px;
                padding-top: 20px;
                border-top: 1px solid #ddd;
                text-align: center;
                color: #7f8c8d;
                font-size: 12px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>NYC Council Meeting Legislative Summary</h1>
            <p>Here is a summary of the items discussed in the recent NYC Council meeting:</p>
    """
    
    for item in items:
        # Determine status class
        status_lower = item['status'].lower().replace(' ', '-')
        status_class = f"status-{status_lower}" if status_lower in ['passed', 'under-review', 'rejected'] else "status-default"
        
        # Convert markdown to basic HTML
        summary_html = convert_markdown_to_html(item['Summarized'])
        
        html += f"""
            <div class="item">
                <div class="item-header">
                    <div class="item-name">{item['name']}</div>
                    <div class="status {status_class}">{item['status']}</div>
                </div>
                <div class="summary">
                    {summary_html}
                </div>
            </div>
        """
    
    html += """
            <div class="footer">
                <p>This is an automated summary from NYC Council Meeting Tracker</p>
                <p>For more information, please visit the official NYC Council website</p>
            </div>
        </div>
    </body>
    </html>
    """
    
    return html

def convert_markdown_to_html(markdown_text):
    """Basic markdown to HTML converter"""
    import re
    
    html = markdown_text
    
    # Headers
    html = re.sub(r'^### (.+)$', r'<h3>\1</h3>', html, flags=re.MULTILINE)
    html = re.sub(r'^## (.+)$', r'<h2>\1</h2>', html, flags=re.MULTILINE)
    html = re.sub(r'^# (.+)$', r'<h1>\1</h1>', html, flags=re.MULTILINE)
    
    # Bold
    html = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', html)
    
    # Lists
    lines = html.split('\n')
    in_list = False
    processed_lines = []
    
    for line in lines:
        if line.strip().startswith('- '):
            if not in_list:
                processed_lines.append('<ul>')
                in_list = True
            processed_lines.append(f'<li>{line.strip()[2:]}</li>')
        else:
            if in_list:
                processed_lines.append('</ul>')
                in_list = False
            processed_lines.append(line)
    
    if in_list:
        processed_lines.append('</ul>')
    
    html = '\n'.join(processed_lines)
    
    # Paragraphs
    html = re.sub(r'\n\n', '</p><p>', html)
    
    return html

### Create HTML email content and retrieve recipients list

In [ ]:
recipients = [
    "hemitvpatel@gmail.com",
    "kush.shah010@gmail.com",
]

def create_html_email(items):
    """Convert markdown summaries to HTML email format"""
    
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                color: #333;
                max-width: 800px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f4f4f4;
            }
            .container {
                background-color: white;
                padding: 30px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            h1 {
                color: #2c3e50;
                border-bottom: 3px solid #3498db;
                padding-bottom: 10px;
            }
            .item {
                margin: 30px 0;
                padding: 20px;
                background-color: #f9f9f9;
                border-left: 4px solid #3498db;
                border-radius: 4px;
            }
            .item-header {
                display: flex;
                justify-content: space-between;
                align-items: center;
                margin-bottom: 15px;
            }
            .item-name {
                font-size: 18px;
                font-weight: bold;
                color: #2c3e50;
                flex: 1;
            }
            .status {
                padding: 5px 15px;
                border-radius: 20px;
                font-size: 14px;
                font-weight: bold;
                text-transform: uppercase;
            }
            .status-passed {
                background-color: #27ae60;
                color: white;
            }
            .status-under-review {
                background-color: #f39c12;
                color: white;
            }
            .status-rejected {
                background-color: #e74c3c;
                color: white;
            }
            .status-default {
                background-color: #95a5a6;
                color: white;
            }
            .summary {
                margin-top: 15px;
                line-height: 1.8;
            }
            .summary h2 {
                color: #2c3e50;
                font-size: 16px;
                margin-top: 15px;
                margin-bottom: 10px;
            }
            .summary h3 {
                color: #34495e;
                font-size: 14px;
                margin-top: 12px;
                margin-bottom: 8px;
            }
            .summary ul {
                margin: 10px 0;
                padding-left: 20px;
            }
            .summary li {
                margin: 5px 0;
            }
            .summary strong {
                color: #2980b9;
            }
            .footer {
                margin-top: 40px;
                padding-top: 20px;
                border-top: 1px solid #ddd;
                text-align: center;
                color: #7f8c8d;
                font-size: 12px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>NYC Council Meeting Legislative Summary</h1>
            <p>Here is a summary of the items discussed in the recent NYC Council meeting:</p>
    """
    
    for item in items:
        # Determine status class
        status_lower = item['status'].lower().replace(' ', '-')
        status_class = f"status-{status_lower}" if status_lower in ['passed', 'under-review', 'rejected'] else "status-default"
        
        # Convert markdown to basic HTML
        summary_html = convert_markdown_to_html(item['Summarized'])
        
        html += f"""
            <div class="item">
                <div class="item-header">
                    <div class="item-name">{item['name']}</div>
                    <div class="status {status_class}">{item['status']}</div>
                </div>
                <div class="summary">
                    {summary_html}
                </div>
            </div>
        """
    
    html += """
            <div class="footer">
                <p>This is an automated summary from Next Voters </p>
                <p>For more information, please visit the official NYC Council website</p>
            </div>
        </div>
    </body>
    </html>
    """
    
    return html

### Send emails to the recipients

In [144]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

recipients = [
    "hemitvpatel@gmail.com",
    "kush.shah010@gmail.com",
]

def send_email(sender_email, sender_password, recipient_email, subject, html_content):
    """Send email using SMTP with MIME"""
    try:
        # Create message
        msg = MIMEMultipart('alternative')
        msg['Subject'] = subject
        msg['From'] = sender_email
        msg['To'] = recipient_email
        
        # Attach HTML content
        html_part = MIMEText(html_content, 'html')
        msg.attach(html_part)
        
        # Connect to Gmail SMTP server
        print(f"Connecting to SMTP server...")
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.set_debuglevel(1)  # Enable debug output
        server.starttls()
        
        print(f"Logging in...")
        server.login(sender_email, sender_password)
        
        print(f"Sending email to {recipient_email}...")
        server.send_message(msg)
        
        print(f"✓ Email sent successfully to {recipient_email}")
        server.quit()
        return True
        
    except smtplib.SMTPAuthenticationError as e:
        print(f"✗ Authentication failed: {e}")
        print("\nTroubleshooting:")
        print("1. Make sure 2-Factor Authentication is enabled on your Google account")
        print("2. Generate an App Password at: https://myaccount.google.com/apppasswords")
        print("3. Use the App Password (16 characters, no spaces) instead of your regular password")
        return False
        
    except Exception as e:
        print(f"✗ Error sending email to {recipient_email}: {e}")
        return False

# Sample data - replace with your actual categories
categories = {
    "Immigration": [
        {
            "Name": "Bill A-123: Immigrant Worker Protection Act",
            "Status": "Passed",
            "Summarized": "## Overview\n\nThis bill provides enhanced protections for immigrant workers.\n\n### Key Points:\n- Prevents wage theft\n- Establishes legal aid fund\n- Creates safe reporting mechanisms"
        }
    ],
    "Economy": [
        {
            "Name": "Resolution B-456: Small Business Tax Relief",
            "Status": "Under Review",
            "Summarized": "## Summary\n\nProposes **tax credits** for small businesses with fewer than 50 employees.\n\n### Main Provisions:\n- 10% tax reduction for eligible businesses\n- Job creation incentives\n- Streamlined application process"
        }
    ],
    "Civil": []
}

# Email configuration
SENDER_EMAIL = "nextvoters@gmail.com"
SENDER_PASSWORD = "oipyupzisrxzfltq "  # Replace with your App Password (no spaces!)
SUBJECT = "NYC Council Meeting Legislative Summary"

# Create HTML email content
print("Generating email content...")
html_content = create_html_email(categories)

# Send to all recipients
print(f"\nSending emails to {len(recipients)} recipients...\n")
success_count = 0
for recipient in recipients:
    if send_email(SENDER_EMAIL, SENDER_PASSWORD, recipient, SUBJECT, html_content):
        success_count += 1
    print()  # Blank line between sends

print(f"\n{'='*50}")
print(f"Summary: {success_count}/{len(recipients)} emails sent successfully")
print(f"{'='*50}")

Generating email content...

Sending emails to 2 recipients...

Connecting to SMTP server...
Logging in...
Sending email to hemitvpatel@gmail.com...


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.90.236.244]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [34.90.236.244]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.90.236.244]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your se

✓ Email sent successfully to hemitvpatel@gmail.com

Connecting to SMTP server...
Logging in...
Sending email to kush.shah010@gmail.com...
✓ Email sent successfully to kush.shah010@gmail.com


Summary: 2/2 emails sent successfully


reply: b'250 2.0.0 OK  1764351612 a640c23a62f3a-b76f5a4860fsm485652966b.63 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1764351612 a640c23a62f3a-b76f5a4860fsm485652966b.63 - gsmtp'
data: (250, b'2.0.0 OK  1764351612 a640c23a62f3a-b76f5a4860fsm485652966b.63 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection a640c23a62f3a-b76f5a4860fsm485652966b.63 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection a640c23a62f3a-b76f5a4860fsm485652966b.63 - gsmtp'
